In [8]:
%load_ext autoreload
%autoreload 2

from py_clob_client.client import ClobClient

import json
import httpx

from clob_model import ClobModel
from gamma_model import GammaModel


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Creating pydantic model

In [9]:
host: str = "https://clob.polymarket.com"
open_client: ClobClient = ClobClient(host=host)

In [28]:
start_cursor = "MA=="
end_cursor = "LTE="
all_markets = []

next_cursor = start_cursor
while next_cursor != "LTE=":
    response = open_client.get_markets(next_cursor=next_cursor)

    markets = response.get("data", [])
    all_markets.extend(markets)

    # Update cursor
    next_cursor = response.get("next_cursor")

In [11]:
len(all_markets)

69643

In [12]:
all_markets[0]

{'enable_order_book': False,
 'active': True,
 'closed': True,
 'archived': False,
 'accepting_orders': False,
 'accepting_order_timestamp': None,
 'minimum_order_size': 15,
 'minimum_tick_size': 0.01,
 'condition_id': '0x9deb0baac40648821f96f01339229a422e2f5c877de55dc4dbf981f95a1e709c',
 'question_id': '0x956f72c01c11615250489a2fcd22dbebdc1d0da86c3f090c28a368dfee714d07',
 'question': 'NFL Saturday: Chiefs vs. Raiders',
 'description': 'In the upcoming NFL game scheduled for January 7, 2023:\n\nIf the Kansas City Chiefs win, this market will resolve to "Chiefs".\n\nIf the Las Vegas Raiders win, this market will resolve to "Raiders".\n\nIf this game ends in a tie, this market will resolve 50-50. If this game is not completed by January 15, 2023, 11:59:59 PM ET, this market will resolve 50-50.',
 'market_slug': 'nfl-saturday-chiefs-vs-raiders',
 'end_date_iso': '2023-01-07T00:00:00Z',
 'game_start_time': '2023-01-07T21:30:00Z',
 'seconds_delay': 3,
 'fpmm': '0xb6577bdC2D74A7A50FA43fB91d7

In [13]:
with open("clob.json", "w") as f:
    json.dump(all_markets[0], f, indent=2)

In [159]:
!datamodel-codegen --input clob.json --input-file-type json --output clob_model.py

In [29]:
for i, item in enumerate(all_markets):
    try:
        market_obj = ClobModel(**item)
    except Exception as e:
        print(f"Validation error at index {i}")
        print(e)
        break

In [30]:
all_markets[6747]["rewards"]

{'rates': [{'asset_address': '0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174',
   'rewards_daily_rate': 1.5}],
 'min_size': 200,
 'max_spread': 3.5}

In [61]:
live_markets = [
    m for m in all_markets
    if m["active"] and not m["closed"] and m["accepting_orders"]
]
live_markets[0]

{'enable_order_book': True,
 'active': True,
 'closed': False,
 'archived': False,
 'accepting_orders': True,
 'accepting_order_timestamp': '2024-11-10T00:04:33Z',
 'minimum_order_size': 5,
 'minimum_tick_size': 0.001,
 'condition_id': '0x290e8e125c5f8523f21500cc59c186f2c23762da51eec212c9dd956e5dabae3b',
 'question_id': '0x2eef1660dff726147edcea90979db633b93121d75a1c95ec122157f6a481c225',
 'question': 'Will Trump repeal Presidential term limits?',
 'description': 'This market will resolve to "Yes" if Donald Trump signs any bill into law that aims to repeal or alter presidential term limits as defined by the 22nd Amendment to the U.S. Constitution, or if the U.S. Supreme Court rules in a way that would permit a president to serve more than two terms by July 31, 2025, at 11:59 PM ET. Otherwise, this market will resolve to "No."\n\nAny law signed meeting these criteria will count, even if its implementation is immediately halted or delayed by judicial or other actions.\n\nThis market\'s r

In [2]:
host_gamma: str = "https://gamma-api.polymarket.com/markets"
limit = 500
offset = 0
all_markets_gamma = []

while True:
    url = f"{host_gamma}?limit={limit}&offset={offset}"
    resp = httpx.get(url)
    data = resp.json()

    if not data:  # empty list means no more markets
        break

    all_markets_gamma.extend(data)
    offset += limit


In [3]:
len(all_markets_gamma)

63360

In [4]:
all_markets_gamma[0]

{'id': '12',
 'question': 'Will Joe Biden get Coronavirus before the election?',
 'conditionId': '0xe3b423dfad8c22ff75c9899c4e8176f628cf4ad4caa00481764d320e7415f7a9',
 'slug': 'will-joe-biden-get-coronavirus-before-the-election',
 'twitterCardImage': 'https://polymarket-static.s3.us-east-2.amazonaws.com/polymarket-twitter-card.png',
 'endDate': '2020-11-04T00:00:00Z',
 'category': 'US-current-affairs',
 'liquidity': '0',
 'image': 'https://polymarket-upload.s3.us-east-2.amazonaws.com/will-joe-biden-get-coronavirus-before-the-election-5a3d4c3b-0a73-419e-a686-be034d2c73ac.png',
 'icon': 'https://polymarket-upload.s3.us-east-2.amazonaws.com/will-joe-biden-get-coronavirus-before-the-election-5a3d4c3b-0a73-419e-a686-be034d2c73ac.png',
 'description': 'This is a market on if presidential candidate Joe Biden will test positive for COVID-19 before November 3rd, 2020. This will be defined as Joe publicly announcing - via any medium, including social media - that he has tested positive for COVID

In [89]:
with open("gamma.json", "w") as f:
    json.dump(all_markets_gamma[0], f, indent=2)

In [90]:
!datamodel-codegen --input gamma.json --input-file-type json --output gamma_model.py

In [7]:
for i, item in enumerate(all_markets_gamma):
    try:
        market_obj = GammaModel(**item)
    except Exception as e:
        print(f"Validation error at index {i}")
        print(e)
        break

Validation error at index 3
1 validation error for GammaModel
liquidityNum
  Input should be a valid integer, got a number with a fractional part [type=int_from_float, input_value=0.18, input_type=float]
    For further information visit https://errors.pydantic.dev/2.11/v/int_from_float


# Making sense of the data

In [31]:
start_cursor = "MA=="
end_cursor = "LTE="
all_markets = []

next_cursor = start_cursor
while next_cursor != "LTE=":
    response = open_client.get_markets(next_cursor=next_cursor)

    markets = response.get("data", [])
    all_markets.extend(markets)

    # Update cursor
    next_cursor = response.get("next_cursor")

In [33]:
len(all_markets)

69643

In [34]:
active_markets = [
    m for m in all_markets
    if m.get("active") is True
    and m.get("closed") is False
    and m.get("accepting_orders") is True
]

In [35]:
len(active_markets)

4492

In [36]:
active_markets[0]

{'enable_order_book': True,
 'active': True,
 'closed': False,
 'archived': False,
 'accepting_orders': True,
 'accepting_order_timestamp': '2024-11-10T00:04:33Z',
 'minimum_order_size': 5,
 'minimum_tick_size': 0.001,
 'condition_id': '0x290e8e125c5f8523f21500cc59c186f2c23762da51eec212c9dd956e5dabae3b',
 'question_id': '0x2eef1660dff726147edcea90979db633b93121d75a1c95ec122157f6a481c225',
 'question': 'Will Trump repeal Presidential term limits?',
 'description': 'This market will resolve to "Yes" if Donald Trump signs any bill into law that aims to repeal or alter presidential term limits as defined by the 22nd Amendment to the U.S. Constitution, or if the U.S. Supreme Court rules in a way that would permit a president to serve more than two terms by July 31, 2025, at 11:59 PM ET. Otherwise, this market will resolve to "No."\n\nAny law signed meeting these criteria will count, even if its implementation is immediately halted or delayed by judicial or other actions.\n\nThis market\'s r

In [69]:
print(active_markets[0]["condition_id"])
print(active_markets[0]["market_slug"])

print(active_markets[0]["active"])
print(active_markets[0]["closed"])
print(active_markets[0]["enable_order_book"])
print(active_markets[0]["accepting_orders"])
print(active_markets[0]["archived"])

0x290e8e125c5f8523f21500cc59c186f2c23762da51eec212c9dd956e5dabae3b
will-trump-repeal-presidential-term-limits
True
False
True
True
False


In [74]:
print(active_markets[0]["description"])

This market will resolve to "Yes" if Donald Trump signs any bill into law that aims to repeal or alter presidential term limits as defined by the 22nd Amendment to the U.S. Constitution, or if the U.S. Supreme Court rules in a way that would permit a president to serve more than two terms by July 31, 2025, at 11:59 PM ET. Otherwise, this market will resolve to "No."

Any law signed meeting these criteria will count, even if its implementation is immediately halted or delayed by judicial or other actions.

This market's resolution source will be official information from the Trump administration and U.S. Supreme Court decisions.


In [73]:
active_markets[20]["tokens"]

[{'token_id': '61673387045681238454585997121170806826401631642653090831500827557099057497566',
  'outcome': 'Yes',
  'price': 0.0225,
  'winner': False},
 {'token_id': '75673530794317709253317435585138828689432689207295322743064114581203964183879',
  'outcome': 'No',
  'price': 0.9775,
  'winner': False}]

In [71]:
active_markets[0]["rewards"]

{'rates': [{'asset_address': '0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174',
   'rewards_daily_rate': 5}],
 'min_size': 50,
 'max_spread': 3.5}